In [4]:

import os
import json
import functools
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
df = pd.read_csv(r"D:\shalini\HS_DataClean_Nov.csv")

In [3]:
# Step 1: Read the config.json file
with open("D:\\Healthscore Codes\\healthscore_dashboard_old1.json", 'r') as file:
    config_data = json.load(file)

In [4]:
# Extract the config_dict from the loaded JSON data
config_dict = config_data['displayname_nature_seq_mapping']

# Create the mappings from the modified structure
display_name_mapping = {}
metric_nature_mapping = {}
metric_seqnum_mapping = {}

# Iterate through the config_dict to populate the mappings
for key, value in config_dict.items():
    display_name_mapping[key] = value[0]
    metric_nature_mapping[key] = value[1]
    metric_seqnum_mapping[key] = value[2]

# Now you have the mappings in their respective dictionaries
print("Display Name Mapping:", display_name_mapping)
print("Metric Nature Mapping:", metric_nature_mapping)
print("Metric Sequence Number Mapping:", metric_seqnum_mapping)

# Step 2: Extract config1 and config2 from the file
level_1_metrics = config_data['level_1_metrics']
level_2_metrics = config_data['level_2_metrics']
level_3_metrics = config_data['level_3_metrics']
level_4_metrics = config_data['level_4_metrics']
level_5_metrics = config_data['level_5_metrics']
feature_groups = config_data['feature_groups']
feature_seqno_mapping = config_data['feature_seqno_mapping']

print("level_1_metrics : ", level_1_metrics)
print("level_2_metrics : ", level_2_metrics)
print("level_3_metrics : ", level_3_metrics)
print("level_4_metrics : ", level_4_metrics)
print("level_5_metrics : ", level_5_metrics)
print("feature_groups : ", feature_groups)
print("feature_seqno_mapping : ", feature_seqno_mapping)



Display Name Mapping: {'DR_Visits': 'Visits', 'Unique Visitors': 'Visitors', 'Page Views': 'Page Views', 'Average Time Spent on Page': 'Average Time Spent (mins)', 'DR_New Visitors': 'New Visitors', 'DR_Server Errors': 'Server Errors', 'DR_Overall Chat': 'Visits to Chat', 'DR_continuing chat from new chat page': 'Continuing chat from new chat page', 'DR_Message Us': 'Visits to Chat (Bot)', 'DR_Agent Chat': 'Visits to Agent Chat (Skip bot)', 'DR_Self Service Initiated': 'Visits to Self Service Features', 'DR_CoxApp | Visits with Chat and No Self Service Initiated': 'Visits with Chat but No Self Service', 'DR_CoxApp | Visits with No Self Service and No Chat': 'Visits with Neither Self Service nor Chat', 'DR_CoxApp | Visits not logged in': 'Not Logged-in Visits', 'DR_Login Page Visits': 'Login Page Visits', 'DR_Successful logins': 'Login Success Rate', 'DR_Auto Logins': 'Share of Auto Login (Remember/Biometrics)', 'DR_Clicks to MFA Verification': 'with TSV', 'DR_Clicks on Forgot User ID':

In [5]:
df['display_names'] = df['Metrics'].map(display_name_mapping)

In [6]:
df['metric_nature'] = df['Metrics'].map(metric_nature_mapping)

In [7]:
df['level_1'] = df['display_names'].apply(lambda x: x if x in level_1_metrics else None)
df['level_2'] = df['display_names'].apply(lambda x: x if x in level_2_metrics else None)
df['level_3'] = df['display_names'].apply(lambda x: x if x in level_3_metrics else None)
df['level_4'] = df['display_names'].apply(lambda x: x if x in level_4_metrics else None)
df['level_5'] = df['display_names'].apply(lambda x: x if x in level_5_metrics else None)

In [8]:
feature_column = 'Unknown'
df['Feature'] = feature_column
for group, metrics in feature_groups.items():
   df['Feature'] = np.where(df['Metrics'].isin(metrics), group, df['Feature'])

In [9]:
def move_row(df, from_idx, to_idx):
    row = df.iloc[from_idx]
    df = df.drop(from_idx).reset_index(drop=True)
    df = pd.concat([df.iloc[:to_idx], row.to_frame().T, df.iloc[to_idx:]]).reset_index(drop=True)
    return df

df = move_row(df, 50, 46)
df = move_row(df, 51, 47)
df = move_row(df, 52, 48)
df = move_row(df, 53, 49)
df = move_row(df, 164, 158)
df = move_row(df, 165, 159)
df = move_row(df, 180, 174)
df = move_row(df, 181, 175)
df = move_row(df, 218, 206)
df = move_row(df, 219, 207)
df = move_row(df, 220, 208)
df = move_row(df, 221, 209)
df = move_row(df, 280, 268)
df = move_row(df, 281, 269)
df = move_row(df, 282, 270)
df = move_row(df, 283, 271)

In [10]:
df['unique_identifier'] = df['Feature'].apply(lambda x: ''.join([word[0].upper() for word in x.split()]))

In [11]:
def generate_hierarchy_id_grouped_by_os(df):
    df['Hierarchy_ID'] = None

   
    df['Feature_Order'] = df.groupby('Operating System Type')['unique_identifier'].transform(lambda x: pd.Series(range(len(x)), index=x.index))

    grouped = df.groupby(['Operating System Type', 'unique_identifier'], group_keys=False)
    results = []

   
    for (os_name, unique_id), group in grouped:
        current_ids = {'level_1': 0, 'level_2': 0, 'level_3': 0, 'level_4': 0, 'level_5': 0}  

        group = group.sort_values('Feature_Order')  

        for index, row in group.iterrows():
            if pd.isna(row['display_names']):
                continue  

            if not pd.isna(row['level_1']):  
                current_ids['level_1'] += 1
                current_ids['level_2'] = 0  
                current_ids['level_3'] = 0  
                current_ids['level_4'] = 0  
                current_ids['level_5'] = 0  
                group.at[index, 'Hierarchy_ID'] = f"{os_name}_{unique_id}_{current_ids['level_1']}"

            elif not pd.isna(row['level_2']):  
                current_ids['level_2'] += 1
                current_ids['level_3'] = 0  
                current_ids['level_4'] = 0  
                current_ids['level_5'] = 0  
                group.at[index, 'Hierarchy_ID'] = f"{os_name}_{unique_id}_{current_ids['level_1']}.{current_ids['level_2']}"

            elif not pd.isna(row['level_3']):  
                current_ids['level_3'] += 1
                current_ids['level_4'] = 0  
                current_ids['level_5'] = 0  
                group.at[index, 'Hierarchy_ID'] = f"{os_name}_{unique_id}_{current_ids['level_1']}.{current_ids['level_2']}.{current_ids['level_3']}"

            elif not pd.isna(row['level_4']):  
                current_ids['level_4'] += 1
                current_ids['level_5'] = 0  
                group.at[index, 'Hierarchy_ID'] = f"{os_name}_{unique_id}_{current_ids['level_1']}.{current_ids['level_2']}.{current_ids['level_3']}.{current_ids['level_4']}"

            elif not pd.isna(row['level_5']):  
                current_ids['level_5'] += 1
                group.at[index, 'Hierarchy_ID'] = f"{os_name}_{unique_id}_{current_ids['level_1']}.{current_ids['level_2']}.{current_ids['level_3']}.{current_ids['level_4']}.{current_ids['level_5']}"

        results.append(group)

   
    df = pd.concat(results, ignore_index=True)

    
    #df = df[df['Display_Names'].notna()]

    
    cols = [col for col in df.columns if col != 'Hierarchy_ID' and col != 'Feature_Order'] + ['Hierarchy_ID']
    return df[cols]


df = generate_hierarchy_id_grouped_by_os(df)


In [12]:
df = generate_hierarchy_id_grouped_by_os(df)

def assign_parent_id(row):
    if not pd.isna(row['level_5']):  
        return f"{row['Hierarchy_ID'].rsplit('.', 1)[0]}"
    elif not pd.isna(row['level_4']):  
        return f"{row['Hierarchy_ID'].rsplit('.', 1)[0]}"
    elif not pd.isna(row['level_3']):  
        return f"{row['Hierarchy_ID'].rsplit('.', 1)[0]}"
    elif not pd.isna(row['level_2']):  
        return f"{row['Hierarchy_ID'].rsplit('.', 1)[0]}"
    elif not pd.isna(row['level_1']): 
        return None
    else:
        return None


df['Parent_ID'] = df.apply(assign_parent_id, axis=1)


In [13]:
df['metric_id'] = range(1, len(df) + 1)

In [14]:
df['metric_sequence_num'] = df.groupby('Operating System Type').cumcount() + 1

In [15]:
df['metric_seqno'] = df['Metrics'].map(metric_seqnum_mapping)

In [16]:
def calculate_level_no(row):
   
    if pd.isna(row['Hierarchy_ID']) or pd.isna(row['Parent_ID']):
        return 1
    
   
    return row['Hierarchy_ID'].count('.') + 1


df['level_no'] = df.apply(calculate_level_no, axis=1)


In [17]:
feature_names = df['Feature'].unique()
feature_mapping = {feature_name: idx + 1 for idx, feature_name in enumerate(feature_names)}

df['feature_id'] = df['Feature'].map(feature_mapping)


In [18]:
# Create 'feature_seq_num' column based on the dictionary
df['feature_seq_num'] = df['Feature'].map(feature_seqno_mapping)

In [19]:
df['create_dt'] = pd.to_datetime('today').normalize().date()

In [20]:
df = df.rename(columns={'Parent_ID': 'parent_id_old'})

In [21]:
def calculate_parent_id(row, df):
    
    if pd.isna(row['Hierarchy_ID']):
        return None
   
    hierarchy_parts = row['Hierarchy_ID'].rsplit('.', 1)
    if len(hierarchy_parts) > 1:
       
        parent_hierarchy = hierarchy_parts[0]
        
        parent_row = df[df['Hierarchy_ID'] == parent_hierarchy]
        if not parent_row.empty:
            return parent_row['metric_id'].values[0]

    return None

df['parent_id'] = df.apply(calculate_parent_id, axis=1, df=df)


df.loc[df['Hierarchy_ID'].isna(), 'parent_id'] = None


In [22]:
df = df.rename(columns={
    'Metrics': 'metrics',
    'Operating System Type': 'operating_system_type',
    'Feature': 'feature_name',
    'Operating System Type': 'operating_system_type',
    'Feature': 'feature_name',
    'Operating System Type': 'operating_system_type',
    'Hierarchy_ID': 'hierarchy_id',
})

In [23]:
df.shape

(463, 111)

In [24]:
print(list(df.columns))

['metrics', 'operating_system_type', '29-08-2024', '30-08-2024', '31-08-2024', '01-09-2024', '02-09-2024', '03-09-2024', '04-09-2024', '05-09-2024', '06-09-2024', '07-09-2024', '08-09-2024', '09-09-2024', '10-09-2024', '11-09-2024', '12-09-2024', '13-09-2024', '14-09-2024', '15-09-2024', '16-09-2024', '17-09-2024', '18-09-2024', '19-09-2024', '20-09-2024', '21-09-2024', '22-09-2024', '23-09-2024', '24-09-2024', '25-09-2024', '26-09-2024', '27-09-2024', '28-09-2024', '29-09-2024', '30-09-2024', '01-10-2024', '02-10-2024', '03-10-2024', '04-10-2024', '05-10-2024', '06-10-2024', '07-10-2024', '08-10-2024', '09-10-2024', '10-10-2024', '11-10-2024', '12-10-2024', '13-10-2024', '14-10-2024', '15-10-2024', '16-10-2024', '17-10-2024', '18-10-2024', '19-10-2024', '20-10-2024', '21-10-2024', '22-10-2024', '23-10-2024', '24-10-2024', '25-10-2024', '26-10-2024', '27-10-2024', '28-10-2024', '29-10-2024', '30-10-2024', '31-10-2024', '01-11-2024', '02-11-2024', '03-11-2024', '04-11-2024', '05-11-2024

In [25]:
first_table_columns = ['metric_id', 'metrics', 'display_names', 'operating_system_type', 'feature_name', 'unique_identifier', 'level_1', 'level_2', 'level_3', 'level_4','level_5', 'hierarchy_id', 'parent_id_old', 'metric_sequence_num', 'metric_seqno', 'level_no', 'feature_id', 'feature_seq_num', 'parent_id', 'metric_nature', 'create_dt' ]
first_table_df = df[first_table_columns]

In [26]:
#first_table_df.to_csv(r'D:\Healthsore Data\table_1_data_all_features.csv', index = False)

In [27]:
def is_date_column(col_name):
    date_formats = ['%d-%m-%Y']
    for fmt in date_formats:
        try:
            pd.to_datetime(col_name, format=fmt)
            return True
        except ValueError:
            continue  # Try the next format
    return False

# Filter columns with date format 'm/d/Y'
date_columns = [col for col in df.columns if is_date_column(col)]

In [28]:
second_table_columns1 = ['metric_id', 'create_dt', 'metrics', 'operating_system_type']
second_table_columns = second_table_columns1 + date_columns
second_table_df = df[second_table_columns]

In [29]:
filtered_date_columns = second_table_df.columns[4:].to_list()

In [30]:
id_vars = ['metric_id', 'create_dt']

In [31]:
second_table_df_melted = second_table_df.melt(id_vars = id_vars, value_vars = filtered_date_columns, var_name = 'Date', value_name = 'Value')

In [32]:
second_table_df_melted.shape

(41670, 4)

In [33]:
#second_table_df_melted.to_csv(r'D:\Healthsore Data\table_2_data_all_features.csv', index = False)

In [34]:
sample_df = df[df['feature_name'].isin(['SMART HELP', 'RESET MODEM', 'RESET TV BOX'])]
sample_df.shape

(68, 111)

In [36]:
def calculate_percentage(df, datecolumns, metricid, parentid):
    # Ensure these columns exist in the DataFrame
    if metricid not in df.columns or parentid not in df.columns:
        raise ValueError(f"'{metricid}' or '{parentid}' columns not found in DataFrame.")
    
    # Create a copy of the DataFrame to avoid setting on a view
    df = df.copy()
    
    # Iterate through each row and calculate the percentage for each DATE column
    for idx, row in df.iterrows():
        # Iterate through all the DATE columns
        for date_col in datecolumns:
            # Get the numerator (DATE value)
            numerator = row[date_col]
            
            # Get the parent hierarchy ID
            parent_id = row[parentid]
            
            # Check if the parent is empty
            if pd.isna(parent_id) or parent_id == "":
                # If the PARENTID is empty, keep the original DATE value
                percentage = numerator  # Assuming 100% when no parent is found
            else:
                # Find the row where HIERARCHYID == parent_hierarchyid to get the parent's DATE value
                parent_row = df[df[metricid] == parent_id]
                
                # If the parent exists, get the parent's DATE value for the current DATE column
                if not parent_row.empty:
                    denominator = parent_row[date_col].values[0]
                else:
                    denominator = 0  # In case no parent is found, avoid division by zero
                
                # Calculate the percentage
                if denominator != 0:
                    percentage = (numerator / denominator) * 100
                else:
                    percentage = 0  # To handle division by zero if no parent found
                
            # Round to 3 decimal places
            percentage = round(percentage, 3)
            
            # Add the percentage to the DataFrame in a new column
            percentage_column_name = f"{date_col} PERCENTAGE"
            if percentage_column_name not in df.columns:
                df[percentage_column_name] = None  # Initialize the column safely
            
            # Set the percentage for the current row and current date column using .loc
            df.loc[idx, percentage_column_name] = percentage

    return df


In [37]:
percentage_df = calculate_percentage(sample_df, date_columns, 'metric_id', 'parent_id')
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 100)
percentage_df.head()

,metrics,operating_system_type,29-08-2024,30-08-2024,31-08-2024,01-09-2024,02-09-2024,03-09-2024,04-09-2024,05-09-2024,06-09-2024,07-09-2024,08-09-2024,09-09-2024,10-09-2024,11-09-2024,12-09-2024,13-09-2024,14-09-2024,15-09-2024,16-09-2024,17-09-2024,18-09-2024,19-09-2024,20-09-2024,21-09-2024,22-09-2024,23-09-2024,24-09-2024,25-09-2024,26-09-2024,27-09-2024,28-09-2024,29-09-2024,30-09-2024,01-10-2024,02-10-2024,03-10-2024,04-10-2024,05-10-2024,06-10-2024,07-10-2024,08-10-2024,09-10-2024,10-10-2024,11-10-2024,12-10-2024,13-10-2024,14-10-2024,15-10-2024,16-10-2024,17-10-2024,18-10-2024,19-10-2024,20-10-2024,21-10-2024,22-10-2024,23-10-2024,24-10-2024,25-10-2024,26-10-2024,27-10-2024,28-10-2024,29-10-2024,30-10-2024,31-10-2024,01-11-2024,02-11-2024,03-11-2024,04-11-2024,05-11-2024,06-11-2024,07-11-2024,08-11-2024,09-11-2024,10-11-2024,11-11-2024,12-11-2024,13-11-2024,14-11-2024,15-11-2024,16-11-2024,17-11-2024,18-11-2024,19-11-2024,20-11-2024,21-11-2024,22-11-2024,23-11-2024,24-11-2024,25-11-2024,26-11-2024,display_names,metric_nature,level_1,level_2,level_3,level_4,level_5,feature_name,unique_identifier,hierarchy_id,parent_id_old,metric_id,metric_sequence_num,metric_seqno,level_no,feature_id,feature_seq_num,create_dt,parent_id,29-08-2024 PERCENTAGE,30-08-2024 PERCENTAGE,31-08-2024 PERCENTAGE,01-09-2024 PERCENTAGE,02-09-2024 PERCENTAGE,03-09-2024 PERCENTAGE,04-09-2024 PERCENTAGE,05-09-2024 PERCENTAGE,06-09-2024 PERCENTAGE,07-09-2024 PERCENTAGE,08-09-2024 PERCENTAGE,09-09-2024 PERCENTAGE,10-09-2024 PERCENTAGE,11-09-2024 PERCENTAGE,12-09-2024 PERCENTAGE,13-09-2024 PERCENTAGE,14-09-2024 PERCENTAGE,15-09-2024 PERCENTAGE,16-09-2024 PERCENTAGE,17-09-2024 PERCENTAGE,18-09-2024 PERCENTAGE,19-09-2024 PERCENTAGE,20-09-2024 PERCENTAGE,21-09-2024 PERCENTAGE,22-09-2024 PERCENTAGE,23-09-2024 PERCENTAGE,24-09-2024 PERCENTAGE,25-09-2024 PERCENTAGE,26-09-2024 PERCENTAGE,27-09-2024 PERCENTAGE,28-09-2024 PERCENTAGE,29-09-2024 PERCENTAGE,30-09-2024 PERCENTAGE,01-10-2024 PERCENTAGE,02-10-2024 PERCENTAGE,03-10-2024 PERCENTAGE,04-10-2024 PERCENTAGE,05-10-2024 PERCENTAGE,06-10-2024 PERCENTAGE,07-10-2024 PERCENTAGE,08-10-2024 PERCENTAGE,09-10-2024 PERCENTAGE,10-10-2024 PERCENTAGE,11-10-2024 PERCENTAGE,12-10-2024 PERCENTAGE,13-10-2024 PERCENTAGE,14-10-2024 PERCENTAGE,15-10-2024 PERCENTAGE,16-10-2024 PERCENTAGE,17-10-2024 PERCENTAGE,18-10-2024 PERCENTAGE,19-10-2024 PERCENTAGE,20-10-2024 PERCENTAGE,21-10-2024 PERCENTAGE,22-10-2024 PERCENTAGE,23-10-2024 PERCENTAGE,24-10-2024 PERCENTAGE,25-10-2024 PERCENTAGE,26-10-2024 PERCENTAGE,27-10-2024 PERCENTAGE,28-10-2024 PERCENTAGE,29-10-2024 PERCENTAGE,30-10-2024 PERCENTAGE,31-10-2024 PERCENTAGE,01-11-2024 PERCENTAGE,02-11-2024 PERCENTAGE,03-11-2024 PERCENTAGE,04-11-2024 PERCENTAGE,05-11-2024 PERCENTAGE,06-11-2024 PERCENTAGE,07-11-2024 PERCENTAGE,08-11-2024 PERCENTAGE,09-11-2024 PERCENTAGE,10-11-2024 PERCENTAGE,11-11-2024 PERCENTAGE,12-11-2024 PERCENTAGE,13-11-2024 PERCENTAGE,14-11-2024 PERCENTAGE,15-11-2024 PERCENTAGE,16-11-2024 PERCENTAGE,17-11-2024 PERCENTAGE,18-11-2024 PERCENTAGE,19-11-2024 PERCENTAGE,20-11-2024 PERCENTAGE,21-11-2024 PERCENTAGE,22-11-2024 PERCENTAGE,23-11-2024 PERCENTAGE,24-11-2024 PERCENTAGE,25-11-2024 PERCENTAGE,26-11-2024 PERCENTAGE
140,DR_Reset Modem - Home Page,Apple iOS,5306.0,5499.0,5415.0,5353.0,5588.0,5238.0,5630.0,6204.0,6118.0,5738.0,5762.0,5322.0,5746.0,4950.0,4861.0,4955.0,5519.0,5737.0,5066.0,4975.0,5269.0,4883.0,5107.0,5354.0,5501.0,4933.0,5254.0,5066.0,5078.0,4948.0,5244.0,5526.0,5932.0,5742.0,5079.0,4916.0,5167.0,5095.0,5282.0,4811.0,4788.0,5203.0,4647.0,4820.0,4883.0,4430.0,4915.0,4860.0,5033.0,4824.0,5393.0,5422.0,5324.0,4951.0,4931.0,4573.0,4797.0,4963.0,4711.0,4791.0,4715.0,4702.0,4636.0,4508.0,4668.0,5393.0,5706.0,4926.0,4612.0,4219.0,4317.0,5308.0,5262.0,5030.0,4957.0,4926.0,4920.0,5601.0,8941.0,5652.0,5370.0,4983.0,4760.0,4617.0,4981.0,5001.0,5260.0,5455.0,4976.0,4739.0,Reset Modem Start Page Visits,Positive,Reset Modem Start Page Visits,None,None,None,None,RESET MODEM,RM

In [38]:
percentage_df.shape

(68, 201)

In [39]:
id_columns = ['create_dt', 'metric_id', 'feature_id', 'feature_seq_num', 'metric_sequence_num', 'metric_seqno','level_no', 'parent_id', 'unique_identifier', 'feature_name', 'metrics', 'display_names', 'operating_system_type', 'metric_nature']

In [40]:
# List of columns that contain "PERCENTAGE"
percentage_columns = [col for col in percentage_df.columns if 'PERCENTAGE' in col]

In [41]:
percent_df = percentage_df[id_columns + percentage_columns]

In [42]:
# Rename columns to remove the word "PERCENTAGE"
percent_df.columns = [col.replace(" PERCENTAGE", "") if "PERCENTAGE" in col else col for col in percent_df.columns]

In [43]:
percent_df.shape

(68, 104)

In [44]:
percent_df.head()

,create_dt,metric_id,feature_id,feature_seq_num,metric_sequence_num,metric_seqno,level_no,parent_id,unique_identifier,feature_name,metrics,display_names,operating_system_type,metric_nature,29-08-2024,30-08-2024,31-08-2024,01-09-2024,02-09-2024,03-09-2024,04-09-2024,05-09-2024,06-09-2024,07-09-2024,08-09-2024,09-09-2024,10-09-2024,11-09-2024,12-09-2024,13-09-2024,14-09-2024,15-09-2024,16-09-2024,17-09-2024,18-09-2024,19-09-2024,20-09-2024,21-09-2024,22-09-2024,23-09-2024,24-09-2024,25-09-2024,26-09-2024,27-09-2024,28-09-2024,29-09-2024,30-09-2024,01-10-2024,02-10-2024,03-10-2024,04-10-2024,05-10-2024,06-10-2024,07-10-2024,08-10-2024,09-10-2024,10-10-2024,11-10-2024,12-10-2024,13-10-2024,14-10-2024,15-10-2024,16-10-2024,17-10-2024,18-10-2024,19-10-2024,20-10-2024,21-10-2024,22-10-2024,23-10-2024,24-10-2024,25-10-2024,26-10-2024,27-10-2024,28-10-2024,29-10-2024,30-10-2024,31-10-2024,01-11-2024,02-11-2024,03-11-2024,04-11-2024,05-11-2024,06-11-2024,07-11-2024,08-11-2024,09-11-2024,10-11-2024,11-11-2024,12-11-2024,13-11-2024,14-11-2024,15-11-2024,16-11-2024,17-11-2024,18-11-2024,19-11-2024,20-11-2024,21-11-2024,22-11-2024,23-11-2024,24-11-2024,25-11-2024,26-11-2024
140,2025-01-29,141,11,6,141,1.0,1,NaN,RM,RESET MODEM,DR_Reset Modem - Home Page,Reset Modem Start Page Visits,Apple iOS,Positive,5306.0,5499.0,5415.0,5353.0,5588.0,5238.0,5630.0,6204.0,6118.0,5738.0,5762.0,5322.0,5746.0,4950.0,4861.0,4955.0,5519.0,5737.0,5066.0,4975.0,5269.0,4883.0,5107.0,5354.0,5501.0,4933.0,5254.0,5066.0,5078.0,4948.0,5244.0,5526.0,5932.0,5742.0,5079.0,4916.0,5167.0,5095.0,5282.0,4811.0,4788.0,5203.0,4647.0,4820.0,4883.0,4430.0,4915.0,4860.0,5033.0,4824.0,5393.0,5422.0,5324.0,4951.0,4931.0,4573.0,4797.0,4963.0,4711.0,4791.0,4715.0,4702.0,4636.0,4508.0,4668.0,5393.0,5706.0,4926.0,4612.0,4219.0,4317.0,5308.0,5262.0,5030.0,4957.0,4926.0,4920.0,5601.0,8941.0,5652.0,5370.0,4983.0,4760.0,4617.0,4981.0,5001.0,5260.0,5455.0,4976.0,4739.0
141,2025-01-29,142,11,6,142,2.0,2,141.0,RM,RESET MODEM,DR_Reset Modem - Started Rebooting Process,Started Reset Process,Apple iOS,Positive,91.199,91.071,91.043,91.612,90.909,90.34,90.515,91.86,91.533,90.519,92.555,91.714,90.637,88.909,90.866,90.817,91.393,91.424,91.453,90.231,91.744,91.214,90.347,91.016,92.129,90.31,91.663,91.196,91.729,90.36,91.686,91.947,92.481,91.414,90.49,90.989,90.981,91.894,91.84,91.561,91.332,91.928,92.232,91.369,91.03,91.67,91.129,90.679,92.271,91.915,91.749,92.088,92.825,93.133,92.213,91.494,91.912,91.416,92.061,91.547,78.876,84.347,93.723,94.476,93.766,93.621,94.532,94.884,93.799,94.098,93.931,86.907,94.584,94.513,93.706,94.316,93.862,94.072,95.068,93.825,93.762,93.859,93.676,93.199,91.207,93.461,93.669,94.042,94.293,93.395
142,2025-01-29,143,11,6,143,3.0,3,142.0,RM,RESET MODEM,DR_Reset Modem - Rebooting Complete,Reset Process Complete,Apple iOS,Positive,30.544,30.551,31.136,31.852,31.28,30.959,27.924,27.479,28.964,31.498,30.677,32.207,31.26,30.834,31.175,31.933,31.761,32.698,31.815,30.14,32.458,29.906,31.491,30.371,32.261,32.727,28.135,33.29,32.675,32.386,31.468,31.982,35.709,32.559,32.05,31.612,31.632,31.269,32.839,32.213,30.848,32.657,34.064,32.016,32.373,33.021,33.512,31.427,33.226,33.175,32.235,32.606,31.971,32.661,32.219,31.286,32.184,33.767,33.825,33.174,32.267,32.703,31.899,31.251,32.122,32.957,31.127,31.579,32.247,32.317,32.281,31.606,29.918,30.164,30.549,30.758,31.247,29.398,42.0,34.339,32.771,31.858,30.433,31.861,30.883,29.397,29.957,29.708,30.754,28.513
143,2025-01-29,144,11,6,144,4.0,3,142.0,RM,RESET MODEM,DR_Reset Modem - Rebooting Incomplete,Reset Process Incomplete,Apple iOS,Negative,45.485,46.745,47.221,46.635,46.024,46.048,51.609,51.43,49.089,47.728,45.753,43.536,45.584,47.217,44.419,45.6,45.341,45.548,45.651,45.556,45.201,47.081,44.928,46.029,44.988,43.434,48.007,44.762,45.019,44.241,46.152,44.302,42.417,44.447,46.584,45.674,44.161,45.237,43.352,44.926,44.935,43.947,43.07,43.937,42.97,42.305,44.787,45.791,43.82,43.054,44.665,45.143,45.467,43.83,44.095,46.08,43.683,42.848,43.21,42.3

In [45]:
def calculate_both(df, metricid, parentid):
    """
    Generate new rows for each unique display_name based on given conditions.

    Args:
    - df (DataFrame): Input DataFrame containing the original data.

    Returns:
    - DataFrame: Updated DataFrame with newly calculated rows.
    """
    new_rows = []  # Store new rows
    seq_num = 1  # Sequential ID for metric_id

    for display_name in df['display_names'].unique():
        # Filter rows for the current display_name
        display_df = df[df['display_names'] == display_name]
            
        # Case 1: If `parent_id_old` has null or empty values
        if display_df[parentid].isnull().any() or display_df[parentid].eq('').any():
            # Sum of `Apple iOS` and `Google Android` for `parent_id_old` being null
            ios_rows = display_df[display_df["operating_system_type"] == "Apple iOS"]
            android_rows = display_df[display_df["operating_system_type"] == "Google Android"]
            
            # Calculate numerator using `where` to avoid downcasting issues
            ios_sums = ios_rows[date_columns].apply(pd.to_numeric, errors='coerce').reset_index(drop=True)
            android_sums = android_rows[date_columns].apply(pd.to_numeric, errors='coerce').reset_index(drop=True)
            numerator = ios_sums + android_sums
            
            # Store numerator in new row, rounded to 3 decimal places
            new_row_values = numerator.sum(axis=0).fillna(0).apply(lambda x: round(x, 3)).to_dict()
        else:
            # Case 2: If `parent_id_old` is not null
            ios_rows = display_df[(display_df["operating_system_type"] == "Apple iOS") & 
                                  (display_df[parentid].notna()) & 
                                  (display_df[parentid] != '')]
            android_rows = display_df[(display_df["operating_system_type"] == "Google Android") & 
                                      (display_df[parentid].notna()) & 
                                      (display_df[parentid] != '')]
            
            # Calculate numerator using `where` to avoid downcasting issues
            ios_sums = ios_rows[date_columns].apply(pd.to_numeric, errors='coerce').reset_index(drop=True)
            android_sums = android_rows[date_columns].apply(pd.to_numeric, errors='coerce').reset_index(drop=True)
            numerator = ios_sums + android_sums
            
            # Filter rows where `parent_id_old` matches `hierarchy_id`
            filtered_data = df[df[metricid].isin(display_df[parentid])]
            ios_data = filtered_data[filtered_data['operating_system_type'] == "Apple iOS"]
            android_data = filtered_data[filtered_data['operating_system_type'] == "Google Android"]

            # Calculate denominator
            ios_sums_denom = ios_data[date_columns].apply(pd.to_numeric, errors='coerce').reset_index(drop=True)
            android_sums_denom = android_data[date_columns].apply(pd.to_numeric, errors='coerce').reset_index(drop=True)
            denominator = ios_sums_denom + android_sums_denom

            # Calculate result as percentage
            result = (numerator / denominator) * 100 if not denominator.empty else pd.DataFrame(0, columns=date_columns, index=numerator.index)
            result = result.round(3)  # Round result to 3 decimal places
            new_row_values = result.sum(axis=0).fillna(0).apply(lambda x: round(x, 3)).to_dict()

        # Create the new row for the current display_name
        new_row = {
            'display_names': display_name,
            'operating_system_type': 'Both',
            'metric_id': f'100000{seq_num}'
        }
        # Add calculated values to the new row
        new_row.update(new_row_values)
        new_rows.append(new_row)
        seq_num += 1

    # Append new rows to the original DataFrame
    new_rows_df = pd.DataFrame(new_rows)
    result_df = pd.concat([df, new_rows_df], ignore_index=True)

    # Reset index
    result_df.reset_index(drop=True, inplace=True)
    return new_rows_df


In [46]:
both_calc_df = calculate_both(sample_df,'metric_id', 'parent_id')
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 300)      # Show 100 rows
both_calc_df.head(5)

,display_names,operating_system_type,metric_id,29-08-2024,30-08-2024,31-08-2024,01-09-2024,02-09-2024,03-09-2024,04-09-2024,05-09-2024,06-09-2024,07-09-2024,08-09-2024,09-09-2024,10-09-2024,11-09-2024,12-09-2024,13-09-2024,14-09-2024,15-09-2024,16-09-2024,17-09-2024,18-09-2024,19-09-2024,20-09-2024,21-09-2024,22-09-2024,23-09-2024,24-09-2024,25-09-2024,26-09-2024,27-09-2024,28-09-2024,29-09-2024,30-09-2024,01-10-2024,02-10-2024,03-10-2024,04-10-2024,05-10-2024,06-10-2024,07-10-2024,08-10-2024,09-10-2024,10-10-2024,11-10-2024,12-10-2024,13-10-2024,14-10-2024,15-10-2024,16-10-2024,17-10-2024,18-10-2024,19-10-2024,20-10-2024,21-10-2024,22-10-2024,23-10-2024,24-10-2024,25-10-2024,26-10-2024,27-10-2024,28-10-2024,29-10-2024,30-10-2024,31-10-2024,01-11-2024,02-11-2024,03-11-2024,04-11-2024,05-11-2024,06-11-2024,07-11-2024,08-11-2024,09-11-2024,10-11-2024,11-11-2024,12-11-2024,13-11-2024,14-11-2024,15-11-2024,16-11-2024,17-11-2024,18-11-2024,19-11-2024,20-11-2024,21-11-2024,22-11-2024,23-11-2024,24-11-2024,25-11-2024,26-11-2024
0,Reset Modem Start Page Visits,Both,1000001,7097.000,7306.000,7188.000,7093.000,7314.000,6942.000,7552.000,8274.000,7993.000,7568.000,7650.000,7100.000,7611.000,6706.000,6428.000,6615.000,7345.000,7793.000,6746.000,6558.000,6989.000,6586.000,6798.000,7125.000,7310.000,6693.000,6919.000,6793.000,6793.000,6566.000,6991.000,7414.000,7974.000,7682.000,6916.000,6714.000,6918.000,6884.000,7109.000,6551.000,6429.000,6972.000,6265.000,6424.000,6616.000,5860.000,6621.000,6584.000,6773.000,6524.000,7276.000,7252.000,7019.000,7073.000,6781.000,6146.000,6416.000,6703.000,6458.000,6364.000,6324.000,6281.000,6361.000,6103.000,6362.000,7202.000,7782.000,6524.000,6172.000,5762.000,5893.000,7219.000,7164.000,6652.000,6693.000,6595.000,6708.000,7621.000,12057.000,7666.000,7169.000,6736.000,6475.000,6336.000,6775.000,6859.000,7164.000,7375.000,6684.000,6443.000
1,Started Reset Process,Both,1000002,91.306,91.213,91.180,91.696,91.045,90.521,90.784,91.624,91.618,90.803,92.902,91.845,90.724,88.741,90.884,90.718,91.681,91.428,91.625,90.378,91.515,91.193,90.482,90.933,92.120,90.811,91.617,91.153,91.874,90.329,91.832,92.015,92.300,91.382,90.558,91.346,91.472,91.996,91.940,91.589,91.476,91.581,92.338,91.469,91.218,91.928,91.270,90.993,91.909,91.876,91.754,91.947,92.435,93.101,92.273,91.930,92.020,91.646,92.165,91.892,78.526,83.936,93.617,94.150,93.964,93.627,94.616,94.513,93.908,94.099,93.959,86.452,94.486,94.708,93.829,94.162,93.888,94.082,95.082,93.739,93.681,93.750,94.008,93.245,90.893,93.687,93.495,93.953,94.195,93.280
2,Reset Process Complete,Both,1000003,30.694,30.222,30.607,31.365,30.996,30.442,27.334,27.397,28.827,31.257,30.083,31.698,30.891,30.331,30.452,31.561,31.155,32.295,30.836,30.083,32.051,29.487,31.247,30.159,31.868,31.853,28.159,32.187,31.998,31.968,30.763,30.944,34.810,31.781,31.646,31.029,31.495,31.107,32.375,31.600,29.944,32.545,33.258,31.399,32.096,32.987,32.351,31.047,32.659,32.649,31.845,31.914,31.628,31.967,31.437,30.637,31.064,33.160,33.485,32.250,31.977,32.473,31.100,31.187,32.335,32.167,30.993,31.252,31.315,31.225,31.335,30.796,29.148,30.540,29.650,31.079,30.867,29.400,40.387,33.440,31.894,30.530,30.031,30.603,30.416,28.229,29.277,29.932,30.289,28.436
3,Reset Process Incomplete,Both,1000004,42.639,43.923,44.583,43.881,42.874,43.730,48.775,48.542,46.306,44.761,43.197,40.853,42.911,43.993,42.349,43.076,42.694,42.049,43.278,42.416,42.433,44.372,42.156,43.479,41.966,41.033,45.149,42.442,42.125,41.662,43.988,41.879,38.886,42.293,43.653,43.209,41.751,41.986,40.483,41.583,42.969,41.597,40.380,41.474,40.199,39.800,42.380,42.864,41.799,41.458,42.106,43.101,43.049,39.499,41.617,43.133,41.955,40.078,40.843,40.065,43.012,42.014,42.485,42.760,42.472,43.719,42.089,42.929,41.770,42.014,41.918,43.278,46.358,44.349,45.255,44.300,43.776,45.370,33.112,44.378,43.627,43.753,44.981,45.311,45.274,46.187,48.328,45.461,44.139,45.075
4,Reset_Modem Interacted with Chat,Both,1000005,14.120,14.631,15.166,14.699,13.516,14.402,15.957,16.85

In [47]:
both_calc_df.shape

(34, 93)

In [48]:
unique_display_names_df_test = df.drop_duplicates(subset='display_names')
unique_display_names_df_test.shape

(180, 111)

In [49]:
id_columns_both_test = ['create_dt', 'feature_id', 'feature_seq_num', 'metric_sequence_num', 'metric_seqno', 'parent_id', 'unique_identifier', 'feature_name', 'metrics', 'display_names','metric_nature' ]
id_column_both_df_test = unique_display_names_df_test[id_columns_both_test]
#id_column_both_df = id_column_both_df[id_column_both_df['feature_name'].isin(['SMART HELP'])]
id_column_both_df_test.shape

(180, 11)

In [50]:
# Perform a left merge on 'display_names'
both_df_test =  pd.merge(both_calc_df, id_column_both_df_test, on='display_names', how='inner')
both_df_test.shape

(34, 103)

In [51]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 300)      # Show 100 rows
both_df_test.head()

,display_names,operating_system_type,metric_id,29-08-2024,30-08-2024,31-08-2024,01-09-2024,02-09-2024,03-09-2024,04-09-2024,05-09-2024,06-09-2024,07-09-2024,08-09-2024,09-09-2024,10-09-2024,11-09-2024,12-09-2024,13-09-2024,14-09-2024,15-09-2024,16-09-2024,17-09-2024,18-09-2024,19-09-2024,20-09-2024,21-09-2024,22-09-2024,23-09-2024,24-09-2024,25-09-2024,26-09-2024,27-09-2024,28-09-2024,29-09-2024,30-09-2024,01-10-2024,02-10-2024,03-10-2024,04-10-2024,05-10-2024,06-10-2024,07-10-2024,08-10-2024,09-10-2024,10-10-2024,11-10-2024,12-10-2024,13-10-2024,14-10-2024,15-10-2024,16-10-2024,17-10-2024,18-10-2024,19-10-2024,20-10-2024,21-10-2024,22-10-2024,23-10-2024,24-10-2024,25-10-2024,26-10-2024,27-10-2024,28-10-2024,29-10-2024,30-10-2024,31-10-2024,01-11-2024,02-11-2024,03-11-2024,04-11-2024,05-11-2024,06-11-2024,07-11-2024,08-11-2024,09-11-2024,10-11-2024,11-11-2024,12-11-2024,13-11-2024,14-11-2024,15-11-2024,16-11-2024,17-11-2024,18-11-2024,19-11-2024,20-11-2024,21-11-2024,22-11-2024,23-11-2024,24-11-2024,25-11-2024,26-11-2024,create_dt,feature_id,feature_seq_num,metric_sequence_num,metric_seqno,parent_id,unique_identifier,feature_name,metrics,metric_nature
0,Reset Modem Start Page Visits,Both,1000001,7097.000,7306.000,7188.000,7093.000,7314.000,6942.000,7552.000,8274.000,7993.000,7568.000,7650.000,7100.000,7611.000,6706.000,6428.000,6615.000,7345.000,7793.000,6746.000,6558.000,6989.000,6586.000,6798.000,7125.000,7310.000,6693.000,6919.000,6793.000,6793.000,6566.000,6991.000,7414.000,7974.000,7682.000,6916.000,6714.000,6918.000,6884.000,7109.000,6551.000,6429.000,6972.000,6265.000,6424.000,6616.000,5860.000,6621.000,6584.000,6773.000,6524.000,7276.000,7252.000,7019.000,7073.000,6781.000,6146.000,6416.000,6703.000,6458.000,6364.000,6324.000,6281.000,6361.000,6103.000,6362.000,7202.000,7782.000,6524.000,6172.000,5762.000,5893.000,7219.000,7164.000,6652.000,6693.000,6595.000,6708.000,7621.000,12057.000,7666.000,7169.000,6736.000,6475.000,6336.000,6775.000,6859.000,7164.000,7375.000,6684.000,6443.000,2025-01-29,11,6,141,1.0,NaN,RM,RESET MODEM,DR_Reset Modem - Home Page,Positive
1,Started Reset Process,Both,1000002,91.306,91.213,91.180,91.696,91.045,90.521,90.784,91.624,91.618,90.803,92.902,91.845,90.724,88.741,90.884,90.718,91.681,91.428,91.625,90.378,91.515,91.193,90.482,90.933,92.120,90.811,91.617,91.153,91.874,90.329,91.832,92.015,92.300,91.382,90.558,91.346,91.472,91.996,91.940,91.589,91.476,91.581,92.338,91.469,91.218,91.928,91.270,90.993,91.909,91.876,91.754,91.947,92.435,93.101,92.273,91.930,92.020,91.646,92.165,91.892,78.526,83.936,93.617,94.150,93.964,93.627,94.616,94.513,93.908,94.099,93.959,86.452,94.486,94.708,93.829,94.162,93.888,94.082,95.082,93.739,93.681,93.750,94.008,93.245,90.893,93.687,93.495,93.953,94.195,93.280,2025-01-29,11,6,142,2.0,141.0,RM,RESET MODEM,DR_Reset Modem - Started Rebooting Process,Positive
2,Reset Process Complete,Both,1000003,30.694,30.222,30.607,31.365,30.996,30.442,27.334,27.397,28.827,31.257,30.083,31.698,30.891,30.331,30.452,31.561,31.155,32.295,30.836,30.083,32.051,29.487,31.247,30.159,31.868,31.853,28.159,32.187,31.998,31.968,30.763,30.944,34.810,31.781,31.646,31.029,31.495,31.107,32.375,31.600,29.944,32.545,33.258,31.399,32.096,32.987,32.351,31.047,32.659,32.649,31.845,31.914,31.628,31.967,31.437,30.637,31.064,33.160,33.485,32.250,31.977,32.473,31.100,31.187,32.335,32.167,30.993,31.252,31.315,31.225,31.335,30.796,29.148,30.540,29.650,31.079,30.867,29.400,40.387,33.440,31.894,30.530,30.031,30.603,30.416,28.229,29.277,29.932,30.289,28.436,2025-01-29,11,6,143,3.0,142.0,RM,RESET MODEM,DR_Reset Modem - Rebooting Complete,Positive
3,Reset Process Incomplete,Both,1000004,42.639,43.923,44.583,43.881,42.874,43.730,48.775,48.542,46.306,44.761,43.197,40.853,42.911,43.993,42.349,43.076,42.694,42.049,43.278,42.416,42.433,44.372,42.156,43.479,41.966,41.033,45.149,42.442,42.125,41.662,43.988,41.879,38.886,42.293,43.653,43.209,41.751,41.986,40.483,41.583,42.969,41.597,40.380,41.474,40.199,39.800,42.380,42.8

In [52]:
print(list(both_df_test))

['display_names', 'operating_system_type', 'metric_id', '29-08-2024', '30-08-2024', '31-08-2024', '01-09-2024', '02-09-2024', '03-09-2024', '04-09-2024', '05-09-2024', '06-09-2024', '07-09-2024', '08-09-2024', '09-09-2024', '10-09-2024', '11-09-2024', '12-09-2024', '13-09-2024', '14-09-2024', '15-09-2024', '16-09-2024', '17-09-2024', '18-09-2024', '19-09-2024', '20-09-2024', '21-09-2024', '22-09-2024', '23-09-2024', '24-09-2024', '25-09-2024', '26-09-2024', '27-09-2024', '28-09-2024', '29-09-2024', '30-09-2024', '01-10-2024', '02-10-2024', '03-10-2024', '04-10-2024', '05-10-2024', '06-10-2024', '07-10-2024', '08-10-2024', '09-10-2024', '10-10-2024', '11-10-2024', '12-10-2024', '13-10-2024', '14-10-2024', '15-10-2024', '16-10-2024', '17-10-2024', '18-10-2024', '19-10-2024', '20-10-2024', '21-10-2024', '22-10-2024', '23-10-2024', '24-10-2024', '25-10-2024', '26-10-2024', '27-10-2024', '28-10-2024', '29-10-2024', '30-10-2024', '31-10-2024', '01-11-2024', '02-11-2024', '03-11-2024', '04-11

In [53]:
both_df_test.shape

(34, 103)

In [54]:
modifiled_first_table_df = pd.concat([df, both_df_test], ignore_index=True)

In [55]:
modifiled_first_table_df.shape

(497, 111)

In [56]:
print(list(modifiled_first_table_df))

['metrics', 'operating_system_type', '29-08-2024', '30-08-2024', '31-08-2024', '01-09-2024', '02-09-2024', '03-09-2024', '04-09-2024', '05-09-2024', '06-09-2024', '07-09-2024', '08-09-2024', '09-09-2024', '10-09-2024', '11-09-2024', '12-09-2024', '13-09-2024', '14-09-2024', '15-09-2024', '16-09-2024', '17-09-2024', '18-09-2024', '19-09-2024', '20-09-2024', '21-09-2024', '22-09-2024', '23-09-2024', '24-09-2024', '25-09-2024', '26-09-2024', '27-09-2024', '28-09-2024', '29-09-2024', '30-09-2024', '01-10-2024', '02-10-2024', '03-10-2024', '04-10-2024', '05-10-2024', '06-10-2024', '07-10-2024', '08-10-2024', '09-10-2024', '10-10-2024', '11-10-2024', '12-10-2024', '13-10-2024', '14-10-2024', '15-10-2024', '16-10-2024', '17-10-2024', '18-10-2024', '19-10-2024', '20-10-2024', '21-10-2024', '22-10-2024', '23-10-2024', '24-10-2024', '25-10-2024', '26-10-2024', '27-10-2024', '28-10-2024', '29-10-2024', '30-10-2024', '31-10-2024', '01-11-2024', '02-11-2024', '03-11-2024', '04-11-2024', '05-11-2024

In [57]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 500)      # Show 100 rows
modifiled_first_table_df = modifiled_first_table_df[['metric_id', 'metrics', 'display_names', 'operating_system_type', 'feature_name', 'unique_identifier', 'metric_nature', 'level_1', 'level_2', 'level_3', 'level_4', 'level_5', 'hierarchy_id', 'parent_id_old', 'metric_sequence_num', 'metric_seqno', 'level_no', 'feature_id', 'feature_seq_num', 'parent_id', 'create_dt']]
modifiled_first_table_df.head()

,metric_id,metrics,display_names,operating_system_type,feature_name,unique_identifier,metric_nature,level_1,level_2,level_3,level_4,level_5,hierarchy_id,parent_id_old,metric_sequence_num,metric_seqno,level_no,feature_id,feature_seq_num,parent_id,create_dt
0,1,DR_Visits to Billing Home,Billing Section Visits,Apple iOS,BIL LING,BL,Positive,Billing Section Visits,None,None,None,None,Apple iOS_BL_1,None,1,1.0,1.0,1,3,NaN,2025-01-29
1,2,DR_Billing Make a Payment,Make Payments,Apple iOS,BIL LING,BL,Positive,None,Make Payments,None,None,None,Apple iOS_BL_1.1,Apple iOS_BL_1,2,2.0,2.0,1,3,1.0,2025-01-29
2,3,DR_Client Errors on Make Payment,Client Errors on Payment Page,Apple iOS,BIL LING,BL,Negative,None,None,Client Errors on Payment Page,None,None,Apple iOS_BL_1.1.1,Apple iOS_BL_1.1,3,5.0,3.0,1,3,2.0,2025-01-29
3,4,DR_Server Errors on Make Payment,Server Errors on Payment Page,Apple iOS,BIL LING,BL,Negative,None,None,Server Errors on Payment Page,None,None,Apple iOS_BL_1.1.2,Apple iOS_BL_1.1,4,6.0,3.0,1,3,2.0,2025-01-29
4,5,DR_Payment Submit,Payment Submit,Apple iOS,BIL LING,BL,Positive,None,Payment Submit,None,None,None,Apple iOS_BL_1.2,Apple iOS_BL_1,5,3.0,2.0,1,3,1.0,2025-01-29


In [58]:
#modifiled_first_table_df.to_csv(r'D:\Healthsore Data\table_1_data_all_featuresnew1.csv', index = False)

In [59]:
unique_display_names_df = sample_df.drop_duplicates(subset='display_names')
unique_display_names_df.shape

(34, 111)

In [60]:
id_columns_both = ['create_dt', 'feature_id', 'unique_identifier', 'feature_name', 'metrics', 'display_names', 'parent_id' ]
id_column_both_df = unique_display_names_df[id_columns_both]
#id_column_both_df = id_column_both_df[id_column_both_df['feature_name'].isin(['SMART HELP'])]
id_column_both_df.shape

(34, 7)

In [61]:
# Perform a left merge on 'display_names'
both_df =  pd.merge(both_calc_df, id_column_both_df, on='display_names', how='inner')
both_df.shape

(34, 99)

In [62]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 300)      # Show 100 rows
both_df.head()


,display_names,operating_system_type,metric_id,29-08-2024,30-08-2024,31-08-2024,01-09-2024,02-09-2024,03-09-2024,04-09-2024,05-09-2024,06-09-2024,07-09-2024,08-09-2024,09-09-2024,10-09-2024,11-09-2024,12-09-2024,13-09-2024,14-09-2024,15-09-2024,16-09-2024,17-09-2024,18-09-2024,19-09-2024,20-09-2024,21-09-2024,22-09-2024,23-09-2024,24-09-2024,25-09-2024,26-09-2024,27-09-2024,28-09-2024,29-09-2024,30-09-2024,01-10-2024,02-10-2024,03-10-2024,04-10-2024,05-10-2024,06-10-2024,07-10-2024,08-10-2024,09-10-2024,10-10-2024,11-10-2024,12-10-2024,13-10-2024,14-10-2024,15-10-2024,16-10-2024,17-10-2024,18-10-2024,19-10-2024,20-10-2024,21-10-2024,22-10-2024,23-10-2024,24-10-2024,25-10-2024,26-10-2024,27-10-2024,28-10-2024,29-10-2024,30-10-2024,31-10-2024,01-11-2024,02-11-2024,03-11-2024,04-11-2024,05-11-2024,06-11-2024,07-11-2024,08-11-2024,09-11-2024,10-11-2024,11-11-2024,12-11-2024,13-11-2024,14-11-2024,15-11-2024,16-11-2024,17-11-2024,18-11-2024,19-11-2024,20-11-2024,21-11-2024,22-11-2024,23-11-2024,24-11-2024,25-11-2024,26-11-2024,create_dt,feature_id,unique_identifier,feature_name,metrics,parent_id
0,Reset Modem Start Page Visits,Both,1000001,7097.000,7306.000,7188.000,7093.000,7314.000,6942.000,7552.000,8274.000,7993.000,7568.000,7650.000,7100.000,7611.000,6706.000,6428.000,6615.000,7345.000,7793.000,6746.000,6558.000,6989.000,6586.000,6798.000,7125.000,7310.000,6693.000,6919.000,6793.000,6793.000,6566.000,6991.000,7414.000,7974.000,7682.000,6916.000,6714.000,6918.000,6884.000,7109.000,6551.000,6429.000,6972.000,6265.000,6424.000,6616.000,5860.000,6621.000,6584.000,6773.000,6524.000,7276.000,7252.000,7019.000,7073.000,6781.000,6146.000,6416.000,6703.000,6458.000,6364.000,6324.000,6281.000,6361.000,6103.000,6362.000,7202.000,7782.000,6524.000,6172.000,5762.000,5893.000,7219.000,7164.000,6652.000,6693.000,6595.000,6708.000,7621.000,12057.000,7666.000,7169.000,6736.000,6475.000,6336.000,6775.000,6859.000,7164.000,7375.000,6684.000,6443.000,2025-01-29,11,RM,RESET MODEM,DR_Reset Modem - Home Page,NaN
1,Started Reset Process,Both,1000002,91.306,91.213,91.180,91.696,91.045,90.521,90.784,91.624,91.618,90.803,92.902,91.845,90.724,88.741,90.884,90.718,91.681,91.428,91.625,90.378,91.515,91.193,90.482,90.933,92.120,90.811,91.617,91.153,91.874,90.329,91.832,92.015,92.300,91.382,90.558,91.346,91.472,91.996,91.940,91.589,91.476,91.581,92.338,91.469,91.218,91.928,91.270,90.993,91.909,91.876,91.754,91.947,92.435,93.101,92.273,91.930,92.020,91.646,92.165,91.892,78.526,83.936,93.617,94.150,93.964,93.627,94.616,94.513,93.908,94.099,93.959,86.452,94.486,94.708,93.829,94.162,93.888,94.082,95.082,93.739,93.681,93.750,94.008,93.245,90.893,93.687,93.495,93.953,94.195,93.280,2025-01-29,11,RM,RESET MODEM,DR_Reset Modem - Started Rebooting Process,141.0
2,Reset Process Complete,Both,1000003,30.694,30.222,30.607,31.365,30.996,30.442,27.334,27.397,28.827,31.257,30.083,31.698,30.891,30.331,30.452,31.561,31.155,32.295,30.836,30.083,32.051,29.487,31.247,30.159,31.868,31.853,28.159,32.187,31.998,31.968,30.763,30.944,34.810,31.781,31.646,31.029,31.495,31.107,32.375,31.600,29.944,32.545,33.258,31.399,32.096,32.987,32.351,31.047,32.659,32.649,31.845,31.914,31.628,31.967,31.437,30.637,31.064,33.160,33.485,32.250,31.977,32.473,31.100,31.187,32.335,32.167,30.993,31.252,31.315,31.225,31.335,30.796,29.148,30.540,29.650,31.079,30.867,29.400,40.387,33.440,31.894,30.530,30.031,30.603,30.416,28.229,29.277,29.932,30.289,28.436,2025-01-29,11,RM,RESET MODEM,DR_Reset Modem - Rebooting Complete,142.0
3,Reset Process Incomplete,Both,1000004,42.639,43.923,44.583,43.881,42.874,43.730,48.775,48.542,46.306,44.761,43.197,40.853,42.911,43.993,42.349,43.076,42.694,42.049,43.278,42.416,42.433,44.372,42.156,43.479,41.966,41.033,45.149,42.442,42.125,41.662,43.988,41.879,38.886,42.293,43.653,43.209,41.751,41.986,40.483,41.583,42.969,41.597,40.380,41.474,40.199,39.800,42.380,42.864,41.799,41.458,42.106,43.101,43.049,39.499,41.617,43.133,41.955,40.078,40.843,40.065,43.012,42.014,42.485,42.760,42.47

In [63]:
third_table_df = pd.concat([percent_df, both_df], ignore_index=True)

In [64]:
third_table_df.shape

(102, 104)

In [65]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 200)      # Show 100 rows
third_table_df.head(5)

,create_dt,metric_id,feature_id,feature_seq_num,metric_sequence_num,metric_seqno,level_no,parent_id,unique_identifier,feature_name,metrics,display_names,operating_system_type,metric_nature,29-08-2024,30-08-2024,31-08-2024,01-09-2024,02-09-2024,03-09-2024,04-09-2024,05-09-2024,06-09-2024,07-09-2024,08-09-2024,09-09-2024,10-09-2024,11-09-2024,12-09-2024,13-09-2024,14-09-2024,15-09-2024,16-09-2024,17-09-2024,18-09-2024,19-09-2024,20-09-2024,21-09-2024,22-09-2024,23-09-2024,24-09-2024,25-09-2024,26-09-2024,27-09-2024,28-09-2024,29-09-2024,30-09-2024,01-10-2024,02-10-2024,03-10-2024,04-10-2024,05-10-2024,06-10-2024,07-10-2024,08-10-2024,09-10-2024,10-10-2024,11-10-2024,12-10-2024,13-10-2024,14-10-2024,15-10-2024,16-10-2024,17-10-2024,18-10-2024,19-10-2024,20-10-2024,21-10-2024,22-10-2024,23-10-2024,24-10-2024,25-10-2024,26-10-2024,27-10-2024,28-10-2024,29-10-2024,30-10-2024,31-10-2024,01-11-2024,02-11-2024,03-11-2024,04-11-2024,05-11-2024,06-11-2024,07-11-2024,08-11-2024,09-11-2024,10-11-2024,11-11-2024,12-11-2024,13-11-2024,14-11-2024,15-11-2024,16-11-2024,17-11-2024,18-11-2024,19-11-2024,20-11-2024,21-11-2024,22-11-2024,23-11-2024,24-11-2024,25-11-2024,26-11-2024
0,2025-01-29,141,11,6,141.0,1.0,1.0,NaN,RM,RESET MODEM,DR_Reset Modem - Home Page,Reset Modem Start Page Visits,Apple iOS,Positive,5306.0,5499.0,5415.0,5353.0,5588.0,5238.0,5630.0,6204.0,6118.0,5738.0,5762.0,5322.0,5746.0,4950.0,4861.0,4955.0,5519.0,5737.0,5066.0,4975.0,5269.0,4883.0,5107.0,5354.0,5501.0,4933.0,5254.0,5066.0,5078.0,4948.0,5244.0,5526.0,5932.0,5742.0,5079.0,4916.0,5167.0,5095.0,5282.0,4811.0,4788.0,5203.0,4647.0,4820.0,4883.0,4430.0,4915.0,4860.0,5033.0,4824.0,5393.0,5422.0,5324.0,4951.0,4931.0,4573.0,4797.0,4963.0,4711.0,4791.0,4715.0,4702.0,4636.0,4508.0,4668.0,5393.0,5706.0,4926.0,4612.0,4219.0,4317.0,5308.0,5262.0,5030.0,4957.0,4926.0,4920.0,5601.0,8941.0,5652.0,5370.0,4983.0,4760.0,4617.0,4981.0,5001.0,5260.0,5455.0,4976.0,4739.0
1,2025-01-29,142,11,6,142.0,2.0,2.0,141.0,RM,RESET MODEM,DR_Reset Modem - Started Rebooting Process,Started Reset Process,Apple iOS,Positive,91.199,91.071,91.043,91.612,90.909,90.34,90.515,91.86,91.533,90.519,92.555,91.714,90.637,88.909,90.866,90.817,91.393,91.424,91.453,90.231,91.744,91.214,90.347,91.016,92.129,90.31,91.663,91.196,91.729,90.36,91.686,91.947,92.481,91.414,90.49,90.989,90.981,91.894,91.84,91.561,91.332,91.928,92.232,91.369,91.03,91.67,91.129,90.679,92.271,91.915,91.749,92.088,92.825,93.133,92.213,91.494,91.912,91.416,92.061,91.547,78.876,84.347,93.723,94.476,93.766,93.621,94.532,94.884,93.799,94.098,93.931,86.907,94.584,94.513,93.706,94.316,93.862,94.072,95.068,93.825,93.762,93.859,93.676,93.199,91.207,93.461,93.669,94.042,94.293,93.395
2,2025-01-29,143,11,6,143.0,3.0,3.0,142.0,RM,RESET MODEM,DR_Reset Modem - Rebooting Complete,Reset Process Complete,Apple iOS,Positive,30.544,30.551,31.136,31.852,31.28,30.959,27.924,27.479,28.964,31.498,30.677,32.207,31.26,30.834,31.175,31.933,31.761,32.698,31.815,30.14,32.458,29.906,31.491,30.371,32.261,32.727,28.135,33.29,32.675,32.386,31.468,31.982,35.709,32.559,32.05,31.612,31.632,31.269,32.839,32.213,30.848,32.657,34.064,32.016,32.373,33.021,33.512,31.427,33.226,33.175,32.235,32.606,31.971,32.661,32.219,31.286,32.184,33.767,33.825,33.174,32.267,32.703,31.899,31.251,32.122,32.957,31.127,31.579,32.247,32.317,32.281,31.606,29.918,30.164,30.549,30.758,31.247,29.398,42.0,34.339,32.771,31.858,30.433,31.861,30.883,29.397,29.957,29.708,30.754,28.513
3,2025-01-29,144,11,6,144.0,4.0,3.0,142.0,RM,RESET MODEM,DR_Reset Modem - Rebooting Incomplete,Reset Process Incomplete,Apple iOS,Negative,45.485,46.745,47.221,46.635,46.024,46.048,51.609,51.43,49.089,47.728,45.753,43.536,45.584,47.217,44.419,45.6,45.341,45.548,45.651,45.556,45.201,47.081,44.928,46.029,44.988,43.434,48.007,44.762,45.019,44.241,46.152,44.302,42.417,44.447,46.584,45.674,44.161,45.237,43.352,44.926,44.935,43.947,43.07,43.937,42.97,42.305,44.787,45.791,43.82,43.054,44.665,45.143,45.467,43.83,44.095,46.08,43.683,42.848,43

In [66]:
#third_table_df.to_csv(r'D:\Healthsore Data\table_3_data_3_features.csv', index = False)

In [67]:
third_table_melted_df = pd.melt(third_table_df, id_vars=id_columns, 
                     value_vars=date_columns, 
                     var_name='Date', 
                     value_name='Value')

In [68]:
third_table_melted_df.shape

(9180, 16)

In [69]:
third_table_melted_df.columns

Index(['create_dt', 'metric_id', 'feature_id', 'feature_seq_num',
       'metric_sequence_num', 'metric_seqno', 'level_no', 'parent_id',
       'unique_identifier', 'feature_name', 'metrics', 'display_names',
       'operating_system_type', 'metric_nature', 'Date', 'Value'],
      dtype='object')

In [70]:
third_table_final_df = third_table_melted_df[['create_dt', 'metric_id', 'feature_id', 'unique_identifier', 'feature_name', 'metrics', 'display_names', 'operating_system_type', 'Date', 'Value']]

In [71]:
third_table_final_df.head()

,create_dt,metric_id,feature_id,unique_identifier,feature_name,metrics,display_names,operating_system_type,Date,Value
0,2025-01-29,141,11,RM,RESET MODEM,DR_Reset Modem - Home Page,Reset Modem Start Page Visits,Apple iOS,29-08-2024,5306.0
1,2025-01-29,142,11,RM,RESET MODEM,DR_Reset Modem - Started Rebooting Process,Started Reset Process,Apple iOS,29-08-2024,91.199
2,2025-01-29,143,11,RM,RESET MODEM,DR_Reset Modem - Rebooting Complete,Reset Process Complete,Apple iOS,29-08-2024,30.544
3,2025-01-29,144,11,RM,RESET MODEM,DR_Reset Modem - Rebooting Incomplete,Reset Process Incomplete,Apple iOS,29-08-2024,45.485
4,2025-01-29,145,11,RM,RESET MODEM,DR_Reset Modem - Agent Chat,Reset_Modem Interacted with Chat,Apple iOS,29-08-2024,14.569


In [72]:
third_table_final_df.shape

(9180, 10)

In [73]:
#third_table_final_df.to_csv(r'D:\Healthsore Data\table_3_data_3_features_athenanew.csv', index = False)

In [74]:
third_table_melted_df.dtypes

create_dt                 object
metric_id                 object
feature_id                 int64
feature_seq_num           object
metric_sequence_num      float64
metric_seqno             float64
level_no                 float64
parent_id                float64
unique_identifier         object
feature_name              object
metrics                   object
display_names             object
operating_system_type     object
metric_nature             object
Date                      object
Value                     object
dtype: object

In [75]:
# Convert 'create_dt' and 'Date' to datetime
third_table_melted_df['create_dt'] = pd.to_datetime(third_table_melted_df['create_dt'], errors='coerce')
third_table_melted_df['Date'] = pd.to_datetime(third_table_melted_df['Date'], errors='coerce')

# Convert 'feature_id', 'metric_sequence_num', 'level_no', 'parent_id' to integers
third_table_melted_df['feature_id'] = third_table_melted_df['feature_id'].astype('Int64')  # 'Int64' to handle missing values (NaN)
third_table_melted_df['feature_seq_num'] = third_table_melted_df['feature_seq_num'].astype('Int64')
third_table_melted_df['metric_sequence_num'] = third_table_melted_df['metric_sequence_num'].astype('Int64')
third_table_melted_df['metric_seqno'] = third_table_melted_df['metric_seqno'].astype('Int64')
third_table_melted_df['level_no'] = third_table_melted_df['level_no'].astype('Int64')
third_table_melted_df['parent_id'] = third_table_melted_df['parent_id'].astype('Int64')
#third_table_melted_df['metric_id'] = third_table_melted_df['metric_id'].astype('Int64')

# Convert 'metric_id', 'unique_identifier', 'feature_name', 'metrics', 'display_names', 'operating_system_type' to strings
third_table_melted_df['metric_id'] = third_table_melted_df['metric_id'].astype(str)
third_table_melted_df['unique_identifier'] = third_table_melted_df['unique_identifier'].astype(str)
third_table_melted_df['feature_name'] = third_table_melted_df['feature_name'].astype(str)
third_table_melted_df['metrics'] = third_table_melted_df['metrics'].astype(str)
third_table_melted_df['display_names'] = third_table_melted_df['display_names'].astype(str)
third_table_melted_df['operating_system_type'] = third_table_melted_df['operating_system_type'].astype(str)
third_table_melted_df['metric_nature'] = third_table_melted_df['metric_nature'].astype(str)

# Convert 'Value' to float
third_table_melted_df['Value'] = pd.to_numeric(third_table_melted_df['Value'], errors='coerce')

C:\Users\gokul.02\AppData\Local\Temp\ipykernel_16896\1602820521.py:3: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  third_table_melted_df['Date'] = pd.to_datetime(third_table_melted_df['Date'], errors='coerce')


In [76]:
third_table_melted_df.dtypes

create_dt                datetime64[ns]
metric_id                        object
feature_id                        Int64
feature_seq_num                   Int64
metric_sequence_num               Int64
metric_seqno                      Int64
level_no                          Int64
parent_id                         Int64
unique_identifier                object
feature_name                     object
metrics                          object
display_names                    object
operating_system_type            object
metric_nature                    object
Date                     datetime64[ns]
Value                           float64
dtype: object

In [77]:
#Calculate 1st and 3rd quartiles
quartiles = third_table_melted_df.groupby(['operating_system_type', 'display_names'])['Value'].quantile([0.25, 0.75]).unstack()
quartiles.reset_index(inplace=True)
quartiles.columns.name = None  # Remove the index name
quartiles.columns = ['operating_system_type', 'display_names', '1st Quartile', '3rd Quartile']
#Calculate IQR
quartiles['IQR'] = quartiles['3rd Quartile'] - quartiles['1st Quartile']

In [78]:
quartiles.shape

(102, 5)

In [79]:
quartiles_iqr_df = pd.merge(third_table_df, quartiles, on=['display_names', 'operating_system_type'], how='outer')

In [80]:
quartiles_iqr_df.shape

(102, 107)

In [81]:
pd.set_option('display.max_columns', None)  # Show all columns 
pd.set_option('display.max_rows', 200)      # Show 100 rows 
quartiles_iqr_df.head()

,create_dt,metric_id,feature_id,feature_seq_num,metric_sequence_num,metric_seqno,level_no,parent_id,unique_identifier,feature_name,metrics,display_names,operating_system_type,metric_nature,29-08-2024,30-08-2024,31-08-2024,01-09-2024,02-09-2024,03-09-2024,04-09-2024,05-09-2024,06-09-2024,07-09-2024,08-09-2024,09-09-2024,10-09-2024,11-09-2024,12-09-2024,13-09-2024,14-09-2024,15-09-2024,16-09-2024,17-09-2024,18-09-2024,19-09-2024,20-09-2024,21-09-2024,22-09-2024,23-09-2024,24-09-2024,25-09-2024,26-09-2024,27-09-2024,28-09-2024,29-09-2024,30-09-2024,01-10-2024,02-10-2024,03-10-2024,04-10-2024,05-10-2024,06-10-2024,07-10-2024,08-10-2024,09-10-2024,10-10-2024,11-10-2024,12-10-2024,13-10-2024,14-10-2024,15-10-2024,16-10-2024,17-10-2024,18-10-2024,19-10-2024,20-10-2024,21-10-2024,22-10-2024,23-10-2024,24-10-2024,25-10-2024,26-10-2024,27-10-2024,28-10-2024,29-10-2024,30-10-2024,31-10-2024,01-11-2024,02-11-2024,03-11-2024,04-11-2024,05-11-2024,06-11-2024,07-11-2024,08-11-2024,09-11-2024,10-11-2024,11-11-2024,12-11-2024,13-11-2024,14-11-2024,15-11-2024,16-11-2024,17-11-2024,18-11-2024,19-11-2024,20-11-2024,21-11-2024,22-11-2024,23-11-2024,24-11-2024,25-11-2024,26-11-2024,1st Quartile,3rd Quartile,IQR
0,2025-01-29,186,15,5,186.0,4.0,4.0,185.0,SH,SMART HELP,DR_SmartHelp Result - Unknown,1st common reason (Unknown),Apple iOS,Negative,78.947,84.0,80.909,68.235,78.049,74.118,73.786,77.622,81.053,77.228,75.455,83.784,88.095,72.727,85.321,82.653,80.435,89.046,60.099,8.108,8.333,7.955,6.667,4.615,7.595,10.938,16.667,9.091,11.628,11.765,10.145,7.921,10.753,8.974,10.667,63.636,75.862,75.789,73.958,75.309,69.663,68.085,74.667,75.714,78.824,60.317,66.667,63.725,57.955,57.647,58.889,54.455,62.195,47.561,61.765,58.095,63.014,54.545,59.211,56.79,65.116,51.25,55.102,54.545,71.429,67.442,65.854,63.095,54.412,66.667,60.0,53.846,57.895,46.479,53.086,65.556,56.164,63.529,70.0,60.0,54.545,55.357,51.351,59.459,55.0,56.962,56.818,62.295,66.142,66.216,54.42275,73.52125,19.09850
1,2025-01-29,10000020,15,NaN,NaN,NaN,NaN,185.0,SH,SMART HELP,DR_SmartHelp Result - Unknown,1st common reason (Unknown),Both,NaN,78.689,80.556,80.392,68.462,76.923,70.29,74.497,76.682,79.221,71.014,74.405,75.214,83.871,76.271,84.516,83.099,77.419,90.612,60.912,11.0,10.345,7.965,6.542,7.407,8.13,9.195,16.176,9.804,11.29,14.286,11.828,8.955,12.5,8.85,8.612,65.254,77.344,69.014,73.485,75.0,66.942,65.854,72.727,70.707,74.167,61.053,66.154,65.909,57.895,58.586,62.295,55.072,61.947,55.085,66.412,58.451,64.356,54.762,57.983,58.182,64.516,49.495,57.246,56.25,71.329,66.387,67.816,64.8,59.574,68.687,54.701,55.455,57.522,47.423,55.446,67.391,57.009,61.789,65.254,57.895,53.153,52.941,49.541,55.446,56.911,57.895,48.701,65.318,61.053,64.151,54.71625,70.60275,15.88650
2,2025-01-29,420,15,5,184.0,4.0,4.0,419.0,SH,SMART HELP,DR_SmartHelp Result - Unknown,1st common reason (Unknown),Google Android,Negative,78.261,72.727,79.07,68.889,74.286,64.151,76.087,75.0,76.271,54.054,72.414,60.465,75.0,82.927,82.609,84.091,68.75,92.754,62.5,19.231,14.815,8.0,6.25,11.628,9.091,4.348,14.706,11.111,10.526,18.919,16.667,12.121,18.519,8.571,3.39,68.293,80.488,55.319,72.222,74.074,59.375,58.621,66.667,58.621,62.857,62.5,64.706,73.333,57.692,64.286,71.875,56.757,61.29,72.222,82.759,59.459,67.857,55.263,55.814,62.069,63.158,42.105,62.5,60.0,71.053,63.636,72.549,68.293,73.077,73.333,40.625,59.375,56.757,50.0,65.0,70.833,58.824,57.895,55.263,53.488,50.0,48.276,45.714,44.444,60.465,60.0,37.879,72.549,50.794,59.375,50.00000,72.13525,22.13525
3,2025-01-29,187,15,5,187.0,5.0,4.0,185.0,SH,SMART HELP,DR_SmartHelp Result - Disconnected,2nd common reason (Disconnected),Apple iOS,Negative,14.474,10.0,11.818,18.824,15.854,15.294,16.505,11.888,10.526,12.871,16.364,8.108,7.143,19.481,11.009,14.286,9.783,9.541,21.182,1.351,0.0,2.273,1.333,0.0,0.0,0.0,0.0,1.515,1.163,0.0,0.0,0.0,0.0,0.0,0.0,10.39,9.195,9.474,16.667,18.519,15.73,13.83,13.333,11.429,11.765,22.222,17.708,22.549,28.409,22.353,22.222,37.624,23.171,26.829,26.471,27.619,26.027,26.1

In [82]:

def calculate_upper_lower(df, parent_column='parent_id'):
    # Ensure parent_column is present in the dataframe
    if parent_column not in df.columns:
        raise ValueError(f"{parent_column} not found in the dataframe")
    
    # Define the function to apply to each row
    def calculate_row(row):
        if pd.isna(row[parent_column]) or row[parent_column] == '':
            # Apply 1.25 logic for rows where parent_id is null, empty, or None
            upper = row['3rd Quartile'] + (1.25 * row['IQR'])
            lower = row['1st Quartile'] - (1.25 * row['IQR'])
        else:
            # Apply 0.75 logic for other rows
            upper = row['3rd Quartile'] + (0.75 * row['IQR'])
            lower = row['1st Quartile'] - (0.75 * row['IQR'])
        
        return pd.Series({'Upper': upper, 'Lower': lower})
    
    # Apply the function row by row
    df[['Upper', 'Lower']] = df.apply(calculate_row, axis=1)

    # Round the Upper column to 1 decimal place
    #df['Upper'] = df['Upper'].round(1)

    return df


In [83]:
upper_lower_df = calculate_upper_lower(quartiles_iqr_df)
#upper_lower_df =upper_lower_df[['display_names','operating_system_type','1st Quartile','3rd Quartile', 'IQR', 'Upper', 'Lower']]
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 200)      # Show 100 rows
upper_lower_df.head()

,create_dt,metric_id,feature_id,feature_seq_num,metric_sequence_num,metric_seqno,level_no,parent_id,unique_identifier,feature_name,metrics,display_names,operating_system_type,metric_nature,29-08-2024,30-08-2024,31-08-2024,01-09-2024,02-09-2024,03-09-2024,04-09-2024,05-09-2024,06-09-2024,07-09-2024,08-09-2024,09-09-2024,10-09-2024,11-09-2024,12-09-2024,13-09-2024,14-09-2024,15-09-2024,16-09-2024,17-09-2024,18-09-2024,19-09-2024,20-09-2024,21-09-2024,22-09-2024,23-09-2024,24-09-2024,25-09-2024,26-09-2024,27-09-2024,28-09-2024,29-09-2024,30-09-2024,01-10-2024,02-10-2024,03-10-2024,04-10-2024,05-10-2024,06-10-2024,07-10-2024,08-10-2024,09-10-2024,10-10-2024,11-10-2024,12-10-2024,13-10-2024,14-10-2024,15-10-2024,16-10-2024,17-10-2024,18-10-2024,19-10-2024,20-10-2024,21-10-2024,22-10-2024,23-10-2024,24-10-2024,25-10-2024,26-10-2024,27-10-2024,28-10-2024,29-10-2024,30-10-2024,31-10-2024,01-11-2024,02-11-2024,03-11-2024,04-11-2024,05-11-2024,06-11-2024,07-11-2024,08-11-2024,09-11-2024,10-11-2024,11-11-2024,12-11-2024,13-11-2024,14-11-2024,15-11-2024,16-11-2024,17-11-2024,18-11-2024,19-11-2024,20-11-2024,21-11-2024,22-11-2024,23-11-2024,24-11-2024,25-11-2024,26-11-2024,1st Quartile,3rd Quartile,IQR,Upper,Lower
0,2025-01-29,186,15,5,186.0,4.0,4.0,185.0,SH,SMART HELP,DR_SmartHelp Result - Unknown,1st common reason (Unknown),Apple iOS,Negative,78.947,84.0,80.909,68.235,78.049,74.118,73.786,77.622,81.053,77.228,75.455,83.784,88.095,72.727,85.321,82.653,80.435,89.046,60.099,8.108,8.333,7.955,6.667,4.615,7.595,10.938,16.667,9.091,11.628,11.765,10.145,7.921,10.753,8.974,10.667,63.636,75.862,75.789,73.958,75.309,69.663,68.085,74.667,75.714,78.824,60.317,66.667,63.725,57.955,57.647,58.889,54.455,62.195,47.561,61.765,58.095,63.014,54.545,59.211,56.79,65.116,51.25,55.102,54.545,71.429,67.442,65.854,63.095,54.412,66.667,60.0,53.846,57.895,46.479,53.086,65.556,56.164,63.529,70.0,60.0,54.545,55.357,51.351,59.459,55.0,56.962,56.818,62.295,66.142,66.216,54.42275,73.52125,19.09850,87.845125,40.098875
1,2025-01-29,10000020,15,NaN,NaN,NaN,NaN,185.0,SH,SMART HELP,DR_SmartHelp Result - Unknown,1st common reason (Unknown),Both,NaN,78.689,80.556,80.392,68.462,76.923,70.29,74.497,76.682,79.221,71.014,74.405,75.214,83.871,76.271,84.516,83.099,77.419,90.612,60.912,11.0,10.345,7.965,6.542,7.407,8.13,9.195,16.176,9.804,11.29,14.286,11.828,8.955,12.5,8.85,8.612,65.254,77.344,69.014,73.485,75.0,66.942,65.854,72.727,70.707,74.167,61.053,66.154,65.909,57.895,58.586,62.295,55.072,61.947,55.085,66.412,58.451,64.356,54.762,57.983,58.182,64.516,49.495,57.246,56.25,71.329,66.387,67.816,64.8,59.574,68.687,54.701,55.455,57.522,47.423,55.446,67.391,57.009,61.789,65.254,57.895,53.153,52.941,49.541,55.446,56.911,57.895,48.701,65.318,61.053,64.151,54.71625,70.60275,15.88650,82.517625,42.801375
2,2025-01-29,420,15,5,184.0,4.0,4.0,419.0,SH,SMART HELP,DR_SmartHelp Result - Unknown,1st common reason (Unknown),Google Android,Negative,78.261,72.727,79.07,68.889,74.286,64.151,76.087,75.0,76.271,54.054,72.414,60.465,75.0,82.927,82.609,84.091,68.75,92.754,62.5,19.231,14.815,8.0,6.25,11.628,9.091,4.348,14.706,11.111,10.526,18.919,16.667,12.121,18.519,8.571,3.39,68.293,80.488,55.319,72.222,74.074,59.375,58.621,66.667,58.621,62.857,62.5,64.706,73.333,57.692,64.286,71.875,56.757,61.29,72.222,82.759,59.459,67.857,55.263,55.814,62.069,63.158,42.105,62.5,60.0,71.053,63.636,72.549,68.293,73.077,73.333,40.625,59.375,56.757,50.0,65.0,70.833,58.824,57.895,55.263,53.488,50.0,48.276,45.714,44.444,60.465,60.0,37.879,72.549,50.794,59.375,50.00000,72.13525,22.13525,88.736688,33.398562
3,2025-01-29,187,15,5,187.0,5.0,4.0,185.0,SH,SMART HELP,DR_SmartHelp Result - Disconnected,2nd common reason (Disconnected),Apple iOS,Negative,14.474,10.0,11.818,18.824,15.854,15.294,16.505,11.888,10.526,12.871,16.364,8.108,7.143,19.481,11.009,14.286,9.783,9.541,21.182,1.351,0.0,2.273,1.333,0.0,0.0,0.0,0.0,1.515,1.163,0.0,0.0,0.0,0.0,0.0,0.0,10.39,9.195,9.474,16.667,18.519,15.73,13.83,13.333,11.429,11.765,22.222,17.708,22

In [84]:
upper_lower_df.shape

(102, 109)

In [86]:
import numpy as np
import pandas as pd

def calculate_last7_and_30_days(df):
    # Extract the date columns (skip the non-date columns like 'display_names' and 'operating_system_type')
    datecolumn = [col for col in df.columns if '-' in col]
    
    # Exclude the latest date column (the first one)
    last_7_columns = datecolumn[-8:-1]  # Get the last 7 columns excluding the latest date
    last_30_columns = datecolumn[-31:-1]

    # Extract the date part and convert them to datetime objects
    date_objects = [pd.to_datetime(col.split()[0], format='%d-%m-%Y') for col in datecolumn]

    # Get the column corresponding to the latest date
    latest_date = max(date_objects)
    latest_date_column = datecolumn[date_objects.index(latest_date)]

    # Create a new column 'Yesterday' with the values from the latest date column
    df.loc[:, 'Yesterday'] = df[latest_date_column]
    
    # Ensure numeric columns before performing mean calculation
    df[last_7_columns] = df[last_7_columns].apply(pd.to_numeric, errors='coerce')
    df[last_30_columns] = df[last_30_columns].apply(pd.to_numeric, errors='coerce')

    # Calculate the mean for each row across the last 7 and 30 date columns
    df.loc[:, 'last_7_days'] = df[last_7_columns].mean(axis=1)
    df.loc[:, 'last_30_days'] = df[last_30_columns].mean(axis=1)

    # Replace 0 and NaN values in 'last_7_days' and 'last_30_days' with NaN to avoid division by zero
    df['last_7_days'] = df['last_7_days'].replace(0, np.nan)
    df['last_30_days'] = df['last_30_days'].replace(0, np.nan)

    # Calculate the percentage change for 'Last 7 Days' with a check for NaN
    df.loc[:, '% Change Last 7 Days'] = np.where(
        df['last_7_days'].isna(), 0, 
        (df['Yesterday'] - df['last_7_days']) / df['last_7_days'] * 100
    )

    # Calculate the percentage change for 'Last 30 Days' with a check for NaN
    df.loc[:, '% Change Last 30 Days'] = np.where(
        df['last_30_days'].isna(), 0, 
        (df['Yesterday'] - df['last_30_days']) / df['last_30_days'] * 100
    )

    # Round the percentage changes to 1 decimal place
    df.loc[:, '% Change Last 7 Days'] = df['% Change Last 7 Days'].round(1)
    df.loc[:, '% Change Last 30 Days'] = df['% Change Last 30 Days'].round(1)
    
    return df


In [87]:
last_7_and_30_days_df  = calculate_last7_and_30_days(upper_lower_df)
#last_7_and_30_days_df = last_7_and_30_days_df[['metric_id', 'metrics', 'display_names', 'operating_system_type', 'feature_name', 'parent_id','Yesterday', 'last_7_days', 'last_30_days', '% Change Last 7 Days', '% Change Last 30 Days']]
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 100)
last_7_and_30_days_df.head()

,create_dt,metric_id,feature_id,feature_seq_num,metric_sequence_num,metric_seqno,level_no,parent_id,unique_identifier,feature_name,metrics,display_names,operating_system_type,metric_nature,29-08-2024,30-08-2024,31-08-2024,01-09-2024,02-09-2024,03-09-2024,04-09-2024,05-09-2024,06-09-2024,07-09-2024,08-09-2024,09-09-2024,10-09-2024,11-09-2024,12-09-2024,13-09-2024,14-09-2024,15-09-2024,16-09-2024,17-09-2024,18-09-2024,19-09-2024,20-09-2024,21-09-2024,22-09-2024,23-09-2024,24-09-2024,25-09-2024,26-09-2024,27-09-2024,28-09-2024,29-09-2024,30-09-2024,01-10-2024,02-10-2024,03-10-2024,04-10-2024,05-10-2024,06-10-2024,07-10-2024,08-10-2024,09-10-2024,10-10-2024,11-10-2024,12-10-2024,13-10-2024,14-10-2024,15-10-2024,16-10-2024,17-10-2024,18-10-2024,19-10-2024,20-10-2024,21-10-2024,22-10-2024,23-10-2024,24-10-2024,25-10-2024,26-10-2024,27-10-2024,28-10-2024,29-10-2024,30-10-2024,31-10-2024,01-11-2024,02-11-2024,03-11-2024,04-11-2024,05-11-2024,06-11-2024,07-11-2024,08-11-2024,09-11-2024,10-11-2024,11-11-2024,12-11-2024,13-11-2024,14-11-2024,15-11-2024,16-11-2024,17-11-2024,18-11-2024,19-11-2024,20-11-2024,21-11-2024,22-11-2024,23-11-2024,24-11-2024,25-11-2024,26-11-2024,1st Quartile,3rd Quartile,IQR,Upper,Lower,Yesterday,last_7_days,last_30_days,% Change Last 7 Days,% Change Last 30 Days
0,2025-01-29,186,15,5,186.0,4.0,4.0,185.0,SH,SMART HELP,DR_SmartHelp Result - Unknown,1st common reason (Unknown),Apple iOS,Negative,78.947,84.0,80.909,68.235,78.049,74.118,73.786,77.622,81.053,77.228,75.455,83.784,88.095,72.727,85.321,82.653,80.435,89.046,60.099,8.108,8.333,7.955,6.667,4.615,7.595,10.938,16.667,9.091,11.628,11.765,10.145,7.921,10.753,8.974,10.667,63.636,75.862,75.789,73.958,75.309,69.663,68.085,74.667,75.714,78.824,60.317,66.667,63.725,57.955,57.647,58.889,54.455,62.195,47.561,61.765,58.095,63.014,54.545,59.211,56.790,65.116,51.250,55.102,54.545,71.429,67.442,65.854,63.095,54.412,66.667,60.000,53.846,57.895,46.479,53.086,65.556,56.164,63.529,70.000,60.000,54.545,55.357,51.351,59.459,55.000,56.962,56.818,62.295,66.142,66.216,54.42275,73.52125,19.09850,87.845125,40.098875,66.216,58.289571,59.206200,13.598365,11.839638
1,2025-01-29,10000020,15,NaN,NaN,NaN,NaN,185.0,SH,SMART HELP,DR_SmartHelp Result - Unknown,1st common reason (Unknown),Both,NaN,78.689,80.556,80.392,68.462,76.923,70.29,74.497,76.682,79.221,71.014,74.405,75.214,83.871,76.271,84.516,83.099,77.419,90.612,60.912,11.0,10.345,7.965,6.542,7.407,8.13,9.195,16.176,9.804,11.29,14.286,11.828,8.955,12.5,8.85,8.612,65.254,77.344,69.014,73.485,75.0,66.942,65.854,72.727,70.707,74.167,61.053,66.154,65.909,57.895,58.586,62.295,55.072,61.947,55.085,66.412,58.451,64.356,54.762,57.983,58.182,64.516,49.495,57.246,56.250,71.329,66.387,67.816,64.800,59.574,68.687,54.701,55.455,57.522,47.423,55.446,67.391,57.009,61.789,65.254,57.895,53.153,52.941,49.541,55.446,56.911,57.895,48.701,65.318,61.053,64.151,54.71625,70.60275,15.88650,82.517625,42.801375,64.151,56.409286,58.837533,13.724184,9.030743
2,2025-01-29,420,15,5,184.0,4.0,4.0,419.0,SH,SMART HELP,DR_SmartHelp Result - Unknown,1st common reason (Unknown),Google Android,Negative,78.261,72.727,79.07,68.889,74.286,64.151,76.087,75.0,76.271,54.054,72.414,60.465,75.0,82.927,82.609,84.091,68.75,92.754,62.5,19.231,14.815,8.0,6.25,11.628,9.091,4.348,14.706,11.111,10.526,18.919,16.667,12.121,18.519,8.571,3.39,68.293,80.488,55.319,72.222,74.074,59.375,58.621,66.667,58.621,62.857,62.5,64.706,73.333,57.692,64.286,71.875,56.757,61.29,72.222,82.759,59.459,67.857,55.263,55.814,62.069,63.158,42.105,62.500,60.000,71.053,63.636,72.549,68.293,73.077,73.333,40.625,59.375,56.757,50.000,65.000,70.833,58.824,57.895,55.263,53.488,50.000,48.276,45.714,44.444,60.465,60.000,37.879,72.549,50.794,59.375,50.00000,72.13525,22.13525,88.736688,33.398562,59.375,53.120714,58.331800,11.773723,1.78839
3,2025-01-29,187,15,5,187.0,5.0,4.0,185.0,SH,SMART HELP,DR_SmartHelp Result - Disconnected,2nd common reason (Disconnected),Apple iOS,Negative,14.474,10.0,11.818,18.824,15.854,15.294,1

In [88]:
last_7_and_30_days_df.shape

(102, 114)

In [89]:
last_7_and_30_days_df[['1st Quartile', '3rd Quartile', 'IQR','Upper', 'Lower', 'Yesterday', 'last_7_days', 'last_30_days', '% Change Last 7 Days', '% Change Last 30 Days']] = last_7_and_30_days_df[['1st Quartile', '3rd Quartile', 'IQR','Upper', 'Lower', 'Yesterday', 'last_7_days', 'last_30_days', '% Change Last 7 Days', '% Change Last 30 Days']].round(3)

In [90]:
last_7_and_30_days_df.head()

,create_dt,metric_id,feature_id,feature_seq_num,metric_sequence_num,metric_seqno,level_no,parent_id,unique_identifier,feature_name,metrics,display_names,operating_system_type,metric_nature,29-08-2024,30-08-2024,31-08-2024,01-09-2024,02-09-2024,03-09-2024,04-09-2024,05-09-2024,06-09-2024,07-09-2024,08-09-2024,09-09-2024,10-09-2024,11-09-2024,12-09-2024,13-09-2024,14-09-2024,15-09-2024,16-09-2024,17-09-2024,18-09-2024,19-09-2024,20-09-2024,21-09-2024,22-09-2024,23-09-2024,24-09-2024,25-09-2024,26-09-2024,27-09-2024,28-09-2024,29-09-2024,30-09-2024,01-10-2024,02-10-2024,03-10-2024,04-10-2024,05-10-2024,06-10-2024,07-10-2024,08-10-2024,09-10-2024,10-10-2024,11-10-2024,12-10-2024,13-10-2024,14-10-2024,15-10-2024,16-10-2024,17-10-2024,18-10-2024,19-10-2024,20-10-2024,21-10-2024,22-10-2024,23-10-2024,24-10-2024,25-10-2024,26-10-2024,27-10-2024,28-10-2024,29-10-2024,30-10-2024,31-10-2024,01-11-2024,02-11-2024,03-11-2024,04-11-2024,05-11-2024,06-11-2024,07-11-2024,08-11-2024,09-11-2024,10-11-2024,11-11-2024,12-11-2024,13-11-2024,14-11-2024,15-11-2024,16-11-2024,17-11-2024,18-11-2024,19-11-2024,20-11-2024,21-11-2024,22-11-2024,23-11-2024,24-11-2024,25-11-2024,26-11-2024,1st Quartile,3rd Quartile,IQR,Upper,Lower,Yesterday,last_7_days,last_30_days,% Change Last 7 Days,% Change Last 30 Days
0,2025-01-29,186,15,5,186.0,4.0,4.0,185.0,SH,SMART HELP,DR_SmartHelp Result - Unknown,1st common reason (Unknown),Apple iOS,Negative,78.947,84.0,80.909,68.235,78.049,74.118,73.786,77.622,81.053,77.228,75.455,83.784,88.095,72.727,85.321,82.653,80.435,89.046,60.099,8.108,8.333,7.955,6.667,4.615,7.595,10.938,16.667,9.091,11.628,11.765,10.145,7.921,10.753,8.974,10.667,63.636,75.862,75.789,73.958,75.309,69.663,68.085,74.667,75.714,78.824,60.317,66.667,63.725,57.955,57.647,58.889,54.455,62.195,47.561,61.765,58.095,63.014,54.545,59.211,56.790,65.116,51.250,55.102,54.545,71.429,67.442,65.854,63.095,54.412,66.667,60.000,53.846,57.895,46.479,53.086,65.556,56.164,63.529,70.000,60.000,54.545,55.357,51.351,59.459,55.000,56.962,56.818,62.295,66.142,66.216,54.423,73.521,19.099,87.845,40.099,66.216,58.290,59.206,13.598365,11.839638
1,2025-01-29,10000020,15,NaN,NaN,NaN,NaN,185.0,SH,SMART HELP,DR_SmartHelp Result - Unknown,1st common reason (Unknown),Both,NaN,78.689,80.556,80.392,68.462,76.923,70.29,74.497,76.682,79.221,71.014,74.405,75.214,83.871,76.271,84.516,83.099,77.419,90.612,60.912,11.0,10.345,7.965,6.542,7.407,8.13,9.195,16.176,9.804,11.29,14.286,11.828,8.955,12.5,8.85,8.612,65.254,77.344,69.014,73.485,75.0,66.942,65.854,72.727,70.707,74.167,61.053,66.154,65.909,57.895,58.586,62.295,55.072,61.947,55.085,66.412,58.451,64.356,54.762,57.983,58.182,64.516,49.495,57.246,56.250,71.329,66.387,67.816,64.800,59.574,68.687,54.701,55.455,57.522,47.423,55.446,67.391,57.009,61.789,65.254,57.895,53.153,52.941,49.541,55.446,56.911,57.895,48.701,65.318,61.053,64.151,54.716,70.603,15.886,82.518,42.801,64.151,56.409,58.838,13.724184,9.030743
2,2025-01-29,420,15,5,184.0,4.0,4.0,419.0,SH,SMART HELP,DR_SmartHelp Result - Unknown,1st common reason (Unknown),Google Android,Negative,78.261,72.727,79.07,68.889,74.286,64.151,76.087,75.0,76.271,54.054,72.414,60.465,75.0,82.927,82.609,84.091,68.75,92.754,62.5,19.231,14.815,8.0,6.25,11.628,9.091,4.348,14.706,11.111,10.526,18.919,16.667,12.121,18.519,8.571,3.39,68.293,80.488,55.319,72.222,74.074,59.375,58.621,66.667,58.621,62.857,62.5,64.706,73.333,57.692,64.286,71.875,56.757,61.29,72.222,82.759,59.459,67.857,55.263,55.814,62.069,63.158,42.105,62.500,60.000,71.053,63.636,72.549,68.293,73.077,73.333,40.625,59.375,56.757,50.000,65.000,70.833,58.824,57.895,55.263,53.488,50.000,48.276,45.714,44.444,60.465,60.000,37.879,72.549,50.794,59.375,50.000,72.135,22.135,88.737,33.399,59.375,53.121,58.332,11.773723,1.78839
3,2025-01-29,187,15,5,187.0,5.0,4.0,185.0,SH,SMART HELP,DR_SmartHelp Result - Disconnected,2nd common reason (Disconnected),Apple iOS,Negative,14.474,10.0,11.818,18.824,15.854,15.294,16.505,11.888,10.526,12.871,16.364,8.108,7.143,19.481,1

In [91]:
last_7_and_30_days_df.shape

(102, 114)

In [92]:
#last_7_and_30_days_df.to_csv(r'D:\Healthsore Data\table_4_data_3_featuresnew.csv', index = False)


In [93]:
fouth_table_columns = ['create_dt','metric_id', 'feature_id', 'unique_identifier', 'feature_name', 'metrics', 'display_names', 'operating_system_type', '1st Quartile', '3rd Quartile', 'IQR', 'Upper', 'Lower', 'Yesterday', 'last_7_days', 'last_30_days', '% Change Last 7 Days', '% Change Last 30 Days']

In [94]:
fourth_table_melted_df = pd.melt(last_7_and_30_days_df, id_vars=fouth_table_columns, 
                     value_vars=date_columns, 
                     var_name='Date', 
                     value_name='Value')

In [95]:
fourth_table_melted_df.shape

(9180, 20)

In [96]:
#fourth_table_melted_df.to_csv(r'D:\Healthsore Data\table_4_data_3_features_athena.csv', index = False)